In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import layers

In [2]:
maxStrands = 6
maxCrossings = 19

In [3]:
knot_data = pd.read_csv('~/Desktop/Math 498/data/knot_volume.csv')

In [4]:
# one hot encoding
# note: better to use built in preprocessing from Keras if possible
#       one-hot encoding is built in but not sure how to do it on a list
#       possibility: multiply each crossing by slot^crossing to get an integer, then one-hot
def one_hot(braid, maxStrands, maxCrossings):

    newBraid = []
    for b in braid:
        if b < 0:
            newBraid.append(b + maxStrands)
        else:
            newBraid.append(b + maxStrands - 1)

    nGenerators = maxStrands * 2

    enc = []

    for crossing in newBraid:
        seq = np.zeros(nGenerators)
        seq[crossing] = 1

        enc = np.append(enc, seq)

    enc = np.append(enc, np.zeros(nGenerators * maxCrossings - len(enc)))

    return enc

In [5]:
# split braids into training and test data
# keep all equivalent braids in same set
train_split = .7

train_data = knot_data.sample(frac=train_split)
test_data = knot_data.drop(train_data.index)

x_train = []
y_train = []

train_braids = train_data["Equivalent Braids"].apply(eval)
train_volumes = train_data["Volume"]

for equivalentBraids, volume in zip(train_braids, train_volumes):
    for braid in equivalentBraids:
        x_train.append(one_hot(braid, maxStrands, maxCrossings))
        y_train.append(volume)

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)


In [6]:
# parse testing data

x_test = []
y_test = []

test_braids = test_data["Equivalent Braids"].apply(eval)
test_volumes = test_data["Volume"]

for equivalentBraids, volume in zip(test_braids, test_volumes):
    for braid in equivalentBraids:
        x_test.append(one_hot(braid, maxStrands, maxCrossings))
        y_test.append(volume)

x_test = np.asarray(x_test)
y_test = np.asarray(y_test)


In [7]:
# network parameters
input_size = (maxStrands * 2) * maxCrossings
hidden_layer_sizes = [100, 100]

In [8]:
# build basic sequential model
model = keras.Sequential()

model.add(keras.Input(input_size,))

for size in hidden_layer_sizes:
    model.add(layers.Dense(size, activation="relu"))

model.add(layers.Dense(1))

model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mean_squared_error"])

In [9]:
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=10
)

Epoch 1/10


2023-04-13 15:40:52.897392: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1041/1041 [==============================] - 1s 690us/step - loss: 14.9082 - mean_squared_error: 14.9082
Epoch 2/10
1041/1041 [==============================] - 1s 590us/step - loss: 8.9233 - mean_squared_error: 8.9233
Epoch 3/10
1041/1041 [==============================] - 1s 586us/step - loss: 7.7919 - mean_squared_error: 7.7919
Epoch 4/10
1041/1041 [==============================] - 1s 582us/step - loss: 7.1612 - mean_squared_error: 7.1612
Epoch 5/10
1041/1041 [==============================] - 1s 582us/step - loss: 6.7823 - mean_squared_error: 6.7823
Epoch 6/10
1041/1041 [==============================] - 1s 697us/step - loss: 6.4838 - mean_squared_error: 6.4838
Epoch 7/10
1041/1041 [==============================] - 1s 663us/step - loss: 6.2034 - mean_squared_error: 6.2034
Epoch 8/10
1041/1041 [==============================] - 1s 593us/step - loss: 5.9682 - mean_squared_error: 5.9682
Epoch 9/10
1041/1041 [==============================] - 1s 666us/step - loss: 5.7150 - mean_squar

In [10]:
results = model.evaluate(x_test, y_test, batch_size=128)

223/223 [==============================] - 0s 456us/step - loss: 8.1369 - mean_squared_error: 8.1369


In [11]:
x_test[0].shape

(228,)

In [12]:
y_test[0]

0.0

In [31]:
model.predict(x_test[0])

ValueError: in user code:

    File "/opt/homebrew/Caskroom/miniforge/base/envs/keras/lib/python3.10/site-packages/keras/engine/training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/keras/lib/python3.10/site-packages/keras/engine/training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/homebrew/Caskroom/miniforge/base/envs/keras/lib/python3.10/site-packages/keras/engine/training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/keras/lib/python3.10/site-packages/keras/engine/training.py", line 1983, in predict_step
        return self(x, training=False)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/keras/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) fro